# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以生活新聞為例，先連結到生活新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E7%94%9F%E6%B4%BB/"  #生活新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [7]:
browser = webdriver.Chrome(executable_path='./chromedriver')
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

while True:

    # Scroll down to the bottom.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load the page.
    time.sleep(2)

    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")

    if new_height == last_height:

        break

    last_height = new_height

### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [8]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [9]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 嘉義市旅館公會買西瓜送醫護　李素珍：力挺防疫最前線

▲嘉義市旅館公會理事長李素珍（右）代表公會買西瓜送給第一線防疫醫護。（圖／記者翁伊森翻攝）
記者翁伊森／嘉義報導
一年一度的鹿草西瓜季節到了！瓜農一度擔心西瓜銷量不好，嘉義市長黃敏惠主動為鹿草西瓜找到出路，先是由嘉義市旅館公會響應「贈西瓜讓醫護人員透心涼」，讓瓜農與防疫期間最辛苦的醫護人員都感受到滿滿的關懷與溫暖。
黃敏惠表示，嘉義市已有確診的病例，站在第一線的醫護人員是最辛苦的一群，既要照顧原有的病患，另一方面也要保護好自身的安全，工作辛勞不在話下。
同樣也是新冠肺炎疫情受影響最嚴重的旅館業者，本著利他的精神，抱持著「人飢已飢、人溺已溺」的態度，慷慨買西瓜贈醫護人員，嘉義市旅館公會理事長李素珍說，雖然旅館業因為這一波疫情「重傷」，業績僅剩二成，但第一線的醫護人員仍為嘉義市守住每一道防線，讓疫情沒有擴大，相信各行各業的業者都感受的到，希望鹿草的西瓜可以讓一整天都穿著悶熱防護衣的醫護人員們「透心涼」，感受到廣大市民的關心、支持與鼓勵。
▲嘉義市旅館公會買西瓜送醫護。（圖／記者翁伊森翻攝）
嘉義市政府建設處表示，嘉南平原是台灣的穀倉，豐富的農產品一直以來都是台灣民眾健康的後盾，面對疫情，防疫不分中央地方，關懷也不分縣市，希望推廣嘉義縣鹿草西瓜，讓更多人擁有健康，才能抵抗病毒。
黃敏惠再次呼籲，這一次的疫情非常嚴峻，但相信每一位市民都是「勇媽」，防疫工作要靠大家一起來，嘉義市政府團隊會陪伴大家共同度過難關。
►有75%酒精，必備必買必囤！！
----------------------------------------------------------------------

[1] 嘉市高粱酒廠變身復古文創園區　4/2-4/5打卡免費體驗復古彈珠台

▲嘉義文創園區前身為台灣第一個高粱酒廠。（圖／記者翁伊森翻攝）
記者翁伊森／嘉義報導
前身為台灣第一個高粱酒廠的嘉義文創園區，改造園區最老的建築物－V棟紅磚屋酒精倉庫，將於4月連假推出好玩、好拍、好學習的「嘉酒柑仔店」，提供民眾學習認識童玩，還可以親臨體驗玩賞。從4月2日到5日連續4天開幕慶，打卡就可以免費體驗復古

▲台北101從3月初起關閉2樓空橋，「僅1樓進出」落實量體溫、酒精消毒。（圖／翻攝自Facebook／TAIPEI 101 MALL 台北101購物中心）
記者蔡惠如／台北報導
台北101購物中心今（3／30）宣布，即將從4／6起縮短營業時間1.5個小時，以因應人力上的調配需求，調整後原上午11時開門營業，將調整至中午12點才開門，此為信義區首間更改營業時間的百貨公司。
台北101購物中心原營業時間是周日至周四為上午11點至晚上9點半；周五至周六為上午11點至晚上10點，4／6起不分平假日調整為中午12點起至晚上9點，營業時間約縮短1.5至２個鐘頭。
台北101表示，其實之前便接到許多廠商提出縮短營業時間要求的討論，以利櫃位人事與工作時間的分配，加上國際其實因為疫情關係，包括泰國、日本等商場也紛紛實施縮短營業時間的措施，以減輕品牌人力成本負擔，更顧及到全館從業人員的健康，經與品牌協商後，決議調整營業時間，今僅先對租戶宣布，4／5將會正式公告。
之前誠品也宣布旗下站前店將從4月起縮短營業時間，另位於大直的美麗新廣場則從3月底起便調整營業時間。
----------------------------------------------------------------------

[11] 高層翹班接機！令下屬接待「染疫子」...害觀光局同仁、兒子都確診　下場慘了

▲觀光局主管被爆出翹班接兒子。（圖／記者李毓康攝，下同）
記者許凱彰／台北報導
新冠肺炎（COVlD-19）疫情逐漸擴散，台灣至30日累積確診306例，5人死亡。日前爆出交通部觀光局某高層主管人員涉指示該局駐機場旅客服務同仁接待其「染疫兒子」返國，還同室喝咖啡，導致該接待員工以及其5歲兒子都雙雙確診新冠肺炎，交通部會同觀光局、桃機公司、內政部移民署等單位進行調查，初步結果出爐。
觀光局桃園機場旅服中心員工（案269）因受到觀光局上屬長官的指示，接待其從菲律賓返國的兒子（案277），30日案269的5歲兒子（案299）也被確診感染新冠病毒肺炎。
▲觀光局同仁到管制區內接待長官兒子。（圖／記者李毓康攝）
對此，交通部指出，觀光局該名主管為該局駐機場旅服中心之「上屬長官」，卻指派下屬單位同仁接待其親友，顯不符公務原則；正值邊境戮力防疫之際，此等「公器私用之情事」，應受嚴厲處分。交通部已指示觀光局將該主管「撤

▲民眾配合登船檢查情形。（圖／航港局提供）
記者賴文萱／台北報導
▲航港局加強宣導乘船應攜帶身分證件。（圖／航港局提供）
航港局強調，船舶如在海上發生事故，在緊急應變、救難及人員身分確認上的困難度均較陸上運輸為高，近年來航港局依據船舶法第11條第2項第7款規定，推動客船應備乘客名冊措施，並致力提升名冊的正確率，已具初步成效。
目前大多數由台灣本島往返離島航線的航商都會確認乘船人員的身分資料，但離島島際間因航程較短，民眾搭船較無隨身攜帶身分證件的習慣。為降低民眾搭乘交通運具產生的防疫風險，航港局已印製海報張貼於海運場站等處所，將全面要求航商加強乘客身分確認，共同維護民眾健康與乘船權益。
▲南海之星2號主要航行於馬公-望安-七美離島航線，肩負澎湖縣望安鄉、七美鄉兩離島海上交通及民生物資運輸任務。（圖／航港局提供）
航港局呼籲，防疫工作，人人有責，雖然疫情期間民眾難免感到不便，期冀在國人的配合下，作好自我防護，使疫情能有效控制，平安渡過肺炎疫情。
----------------------------------------------------------------------

[22] 粉鼻萌貓「寄養→收編」超愛玩拋接　她相處5個月傻眼：養到狗？

記者陳靜／採訪報導
到底是養到什麼了啦！飼主Sandy Wang先前在因緣際會之下，收編了友人寄養的粉鼻小萌貓「貢丸」，時間過得非常快，毛孩現在已經7個月大了，但是馬麻近期卻越來越覺得貢丸根本不像是隻貓咪，尤其特愛玩你丟我撿和咬娃娃的牠，讓媽媽數度懷疑自己是不是「養到狗」了！
►寄養十幾天「睡出感情」　她痛失愛兔...小貓「搭手安慰」結局超暖
▲貢丸超愛玩你丟我撿，媽媽以為養到狗。（圖／網友Sandy Wang授權提供，下同）
就在馬麻意外發現了毛孩這個技能後，每天回家就得陪牠玩你丟我撿才行，讓她哭笑不得直呼，「真是懷疑我到底是養到猫還是養到狗了...哈哈。」除此之外，貢丸也很喜歡咬著牠的小人娃娃到處跑，牠甚至還會跟家人對話，隨著不同的心情發出不同的聲音，「牠的叫聲超好玩，有分塞奶型和碎碎唸型的。」
▲貢丸平時無厘頭的行為都讓家人笑翻。
日前媽媽帶著貢丸去結紮，沒想到牠卻記仇記了一整天，回家路上全程都對著馬麻哈氣碎唸，一到家看見把拔時，還自己跑過去唸了一大堆，就像是在跟他抱怨討拍一樣，讓全家人都笑翻。看著時而撒嬌、

▲晶華酒店也推出「保障家人‧安心LONG STAY」優惠住房專案，只要3,800元即可入住13坪豪華客房。（示意圖／晶華酒店提供）
記者彭懷玉／台北報導
----------------------------------------------------------------------

[32] 27年前體檢！郝柏村「頭骨殘留子彈碎片」　曝戰場生死一瞬...軍機掃射滿頭血

▲郝柏村堪稱橫跨台灣黨政、軍旅生涯第一人。（圖／記者邱顯燁翻攝）
記者周亭瑋／綜合報導
前行政院長郝柏村中風引發器官衰竭，30日於三總逝世，享嵩壽102歲，為傳奇人生畫下句點。一生奉獻軍旅的郝前院長，是「真槍實彈」站前線面對大大小小的戰役，他曾透露，1993年體檢時，發現頭骨竟殘留子彈碎片，更回憶起當年無情砲火下，激烈的前線戰況。
17歲便從軍的郝柏村，經歷抗日戰爭、國共內戰、國民黨遷台及823砲戰，也曾赴美國陸軍指揮參謀學院深造，他更是中華民國國軍在位最久的參謀總長，退伍後歷任國防部部長、行政院院長，甚至曾脫黨參選副總統。
▲823砲戰爆發，時任國防會議副秘書長的蔣經國乘船到小金門，聽取時任陸軍第九師師長郝柏村的作戰簡報。（圖／取自國防部期刊）
1938年底，郝柏村參加廣州戰役，怎料支援防禦日本軍隊的進攻時，在未被通知緊急撤退的情況下，車隊遭日軍戰機掃射，他身旁的駕駛當場身亡，而他則滿頭鮮血，因此負傷。
他在《郝柏村回憶錄》提到，1993年他赴三軍總醫院體檢時，透過電腦斷層掃描才赫然發現，左側頭骨上竟殘留一片小小的子彈碎片。而與之和平共存超過35年的金屬碎片，早已被增生組織包住，並無任何影響，所以郝也全然沒有開刀的打算。
當時，不僅醫護人員覺得驚奇，郝柏村更開玩笑地說，他到老年階段都沒有出現失智問題，或許要感謝這塊碎彈片。
----------------------------------------------------------------------

[33] 確診不到7天過世！　名醫曝志村健「驟逝4關鍵」：日本檢疫疏忽害的

▲志村健驟逝可能跟日本當局檢疫速度太慢有關。（圖／翻攝自推特）
記者林妤柔／採訪報導
日本喜劇天王志村健感染新冠肺炎病逝，享壽70歲，成為日本演藝圈首位確診後過世的人。胸腔內科醫師表示，志村健從確診到病逝不到一週，可能是早已感染卻沒馬上診斷，最終

實習記者呂欣潞／採訪報導
▲橘子脾氣超好，最後溫柔掙脫妹妹。
飼主曾小姐接受《ETtoday寵物雲》採訪時表示，這天是8公斤的橘子躺在沙發上休息，10公斤的寶寶硬要去抱抱，牠費了一番功夫才勉強逃離魔爪，最後還回過頭聞聞妹妹身上的味道，「橘子平時被抓住都會碎碎念幾聲，但不會生氣，是好脾氣大暖男！」
▲6歲的橘子是個暖男！
平時妹妹清醒時，家中毛孩子都會避免被逮到，但是發現寶寶睡著後牠們就會默默圍過去觀察！曾小姐分享，雖然家中貓咪都是撿回來的浪浪，但是牠們都很聰明且富有靈性，還會守護自己的小妹妹，畫面相當溫馨！
▲毛孩們默默地守護妹妹睡覺。
 
▲妹妹的童年有狗狗貓咪的陪伴好幸福！
【你可能也想看】
►離世貓託夢：玩幾天就回家　43歲媽意外懷孕見「熟悉睡姿」哭了
►愛犬異常吠叫「滿嘴口水」迎飼主返家！　見重摔影像痛哭失聲
 
►路中間啃芭樂！北投獼猴「握手回應」好心男　專家解答真正含意
 
▼更多精彩內容。
----------------------------------------------------------------------

[45] 調皮爸餵食「空氣零食」！　呆萌阿金受騙秒擺撲克臉

實習記者鄭暐儒／綜合報導
▲阿寶被把拔的空氣零食騙啦！（圖／粉專阿寶日記授權提供）
阿寶是隻貪吃的阿金，但在吃東西前都會十分守規矩，不論是正餐還是點心，都會等把拔數123時，才會開動享用美食。「這天閒來無事，想說如果手上沒有拿任何東西數123的話，阿寶會不會也吃呢？」阿寶爸調皮的說道，影片中，把拔用手指掐出了一個零食的形狀，並把手放到阿寶眼前倒數3秒；當把拔一數完，阿寶張大嘴一口含住了把拔手，發現被騙後，便立刻收起笑容，擺出受騙的撲克臉看向鏡頭。
▲阿寶：「把拔，我們說好的信任跟零食呢？」（圖／粉專阿寶日記授權提供）
►ETtoday寵物雲有IG了，快按追蹤
「偶爾我們會給牠一些零食，但最近覺得牠有一點點變胖了，所以想說試試空氣零食！」阿寶爸哭笑不得的說道，也因為阿寶小吃貨的個性，讓把拔十分篤定傻傻的阿寶會中計；然而，在看到阿寶可愛的反應後，讓飼主忍不住以相同的把戲再次捉弄牠，沒想到，受騙過的阿寶仍再次上當，讓飼主也忍不住笑說，「怎麼這麼呆萌好騙，實在有夠可愛啦！」
影片分享到臉書粉專後，阿寶傻呼呼的反應也讓網友們紛紛融化，並留言說，「阿寶果然沒讓人失望XD」、「

記者黃可昀／綜合報導
前行政院長郝柏村因為中風引發器官衰竭，於今日14時47分在三軍總醫院辭世。白色正義聯盟表示，雖然大家對於郝柏村的生平事蹟都有不同解讀，但不可否認的是，「當年823砲戰，國共對立跟戒嚴時期，沒有這幫軍人守著，台灣早沒了。」
白色正義聯盟在臉書粉專PO文，不少人為郝柏村貼上了「媚共」的標籤，事實上，他是真的站在第一線和軍人並肩作戰的「一代傳奇」，且國軍在郝柏村時期戰力達到巔峰，博得不少人的景仰和敬佩。一名網友也在PTT表示，當年「823砲戰重挫共軍」，郝柏村因而獲得雲麾勳章與虎字榮譽旗，他是「真正跟中共火拼過的男人」，與平時在鍵盤上唇槍舌戰的網友不一樣，而且他直到離開人世前，「都堅持不承認中共政權的合法性」。
據了解，郝柏村生前不僅獲頒雲麾勳章、虎字榮譽旗，還得到卿雲勳章和青天白日勳章的認可，在823砲戰期間擔任最前線小金門陸軍步兵第九師師長，還曾遠赴美國陸軍指揮參謀學院深造，是中華民國留美將領之一，更是我國在位最久的參謀總長。
郝柏村17歲便從軍，歷經了抗日戰爭、國共內戰、國民黨遷台籍823砲戰，見證了過去100年大大小小的關鍵時刻，就連晚年仍以90歲高齡前往大陸抗日戰場，努力還原由國民政府領導對日抗戰的史實。
此外，考量到近日疫情非常時期，郝龍斌配合政府避免群聚的建議，目前暫時婉謝各界前往弔唁，待日後疫情趨緩，將再擇日舉辦告別式。
----------------------------------------------------------------------

[57] 一個超級傳播者能毀掉國家！若台灣有一萬人確診...

● 謝宗學／中山醫學大學附設醫院兒童急診科主任
▲台灣前235位新冠肺炎病例分析。（圖／謝宗學醫師提供）
到3月26日為止，解隔離出院人數只有28位 (12%)，主要原因是，發病後到檢體轉為陰性的時間過長。發病後製第一套檢體轉為陰性時間，是3到49天（中位數是19.5天）；發病後到第二／第三套檢體轉為陰性的時間，是8到53天 (中位數是 24 天)。
目前出院需要達到連續三套檢體陰性，訂定如此嚴謹出院條件，主要是擔心痊癒後出院後，仍然有傳染力讓他人發病，對於新冠肺炎病毒特性尚未完全明瞭的緣故。
▲當台灣爆發社區傳染，一萬人染病，醫療體系恐崩潰。（圖／記者陳凱力攝）
若爆發大規模社區感染　病床將不敷使用
以上台灣確

▲第三架武漢台商包機昨晚間抵台，共有153名乘客。（圖／記者李毓康攝）
記者趙于婷／台北報導
第三架武漢台商包機昨晚間抵台，共有153名乘客，中央流行疫情指揮中心也安排相關採檢工作。指揮官陳時中今（30）日指出，153名台商都有做採檢，但其中3名檢體品質不佳，所以要重新採檢，目前已經全數安排在國內一處檢疫所，將進行14天集中檢疫。
第三架包機比照先前班機消毒模式，華航編號B-18002班機降落機場後即停進廠棚，返台國人配合高規格檢疫後，完成入境手續並隔離安置，過程除了移民署、防檢局、航警局等單位外，陸軍第六軍團33化學兵群也謹慎為人員及機具執行消毒。而該批乘客也在今凌晨1點全部進入檢疫所內，其中3名要重新採檢的乘客也已安排採檢。
根據指揮中心統計，昨日國內新增867例新型冠狀病毒肺炎相關通報，截至目前累計30,777例（含27,378例排除），其中306例確診（今日新增案299至306），分別為261例境外移入及45例本土病例；確診個案中5人死亡，39人解除隔離，其餘持續住院隔離中。
另外，依國際監測資料顯示，全球累計703,185例確診，分布於177個國家/地區；病例數以美國137,294例、義大利97,689例、中國大陸81,470例、西班牙78,797例及德國52,547例為多；病例中33,434例死亡，以義大利10,779例、西班牙6,528例、中國大陸3,304例、伊朗2,640例及法國2,606例為多。
指揮中心再次提醒，民眾應做好手部衛生與咳嗽禮節，自國外入境時，如有發燒、咳嗽等不適症狀，應主動通報機場及港口檢疫人員並配合防疫措施；返國後應落實14天居家檢疫，如出現疑似症狀，請聯繫衛生局或各縣市關懷中心，並依指示就醫，切勿搭乘大眾運輸工具；就醫時請務必告知醫師旅遊史、職業別、接觸史及是否群聚（TOCC），以及時診斷通報。
----------------------------------------------------------------------

[67] 南投同富國中柔道美少女！　獲家扶基金全國自強兒少表揚

▲南投縣同富國中代理校長許淑春（右一）與南投家扶中心主任魏季李（左一），頒贈全國自強兒童獎項，肯定蔻蔻霂．伊斯卡卡夫特。（圖／南投家扶中心提供）
記者張弘昌／南投報導
南投同富國中三年級原住民族學生蔻蔻霂．伊斯卡卡夫特，以柔道運